# Init

In [1]:
import pandas  as pd
import pickle
import matplotlib.pyplot
import sys
import numpy as np
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree
import matplotlib.pyplot as plt
from functools import reduce
from tqdm import tqdm
import  snakeviz

import itertools

In [2]:
sys.path.append('C:\Repos\WhoCell\who_cell')
sys.path.append('C:\Repos\WhoCell')
sys.path.append('C:/Repos/WhoCell/who_cell/simulation/')
sys.path.append('C:\Repos\WhoCell\who_cell\experiments')
sys.path.append('C:/Repos/WhoCell/who_cell/models')
sys.path.append('C:\Repos\pomegranate')


from pomegranate import *
import transitions_dict
import who_cell
from gibbs_experiments import GibbsExperiment
from gibbs_sampler import GibbsSampler
from simulation_for_gibbs import Simulator_for_Gibbs

In [3]:
from hmmlearn import hmm

# Utils

In [4]:
def extrect_states_transitions_dict_from_pome_model(model,states = None) : 
    if states is None : 
        states = model.get_params()['states']
    
    edges = model.get_params()['edges']
    transition_dict = {}
    final_states = [] 
    for e in edges : 
        if ('start' in  states[e[0]].name)   :
            continue
        if ('end' in  states[e[1]].name) : 
            final_states.append(eval(states[e[0]].name))
            continue

        _from = eval(states[e[0]].name)
        _to = eval(states[e[1]].name)
        _weight = e[2]

        if _from not in transition_dict.keys() : 
            transition_dict[_from] = {_to : _weight}
        else : 
            transition_dict[_from][_to] = _weight
    return transition_dict, final_states 

def get_transition(_f,_t,trans_dict):
    if _f in trans_dict.keys():
        if _t in trans_dict[_f].keys():
            return trans_dict[_f][_t]
    return 0

# validation

In [5]:
N=5
d = 4
number_of_states = 8
mues = list(range(1,number_of_states+1))
sigmas = [m/10 for m in mues]

In [6]:
# build pome model for test
_simulator = Simulator_for_Gibbs(N,d,number_of_states,500,1,True)
pome_meta =  _simulator.build_pome_model(N, d, mues, sigmas,True)
states_name_to_mue = pome_meta[-1]


pome_model = pome_meta[0]
transitions  = extrect_states_transitions_dict_from_pome_model(pome_model)[0]


In [7]:
pome_model.log_probability(pome_model.sample())

-8.227331054411664

In [85]:
# build hmmlearn model
_mues_hlearn = np.zeros((N*d,1))
_trans_hlearn = np.zeros((N*d,N*d))
_inital_hlearn = np.zeros(N*d)

_inital_hlearn[:d] = 1/d
for _d,_t in itertools.product(range(d),range(N)) : 
    _loc = _t*(N-1) + _d
    _mues_hlearn[_loc] = states_name_to_mue[((_d,_t))] +1 

# _cov_hlearn = _mues_hlearn/10
_cov_hlearn = np.array([np.array([mu/10]) for mu in _mues_hlearn])

for _d,_t in itertools.product(range(d),range(N)) : 
    for _dd,_tt in itertools.product(range(d),range(N)) :
        _from_loc = _t*(N-1) + _d
        _to_loc = _tt*(N-1) + _dd
        
        _trans_hlearn[_from_loc,_to_loc] = get_transition((_d,_t),(_dd,_tt),transitions)


In [86]:
_cov_hlearn

array([[[0.7]],

       [[0.3]],

       [[0.6]],

       [[0.1]],

       [[0.2]],

       [[0.6]],

       [[0.7]],

       [[0.5]],

       [[0.4]],

       [[0.6]],

       [[0.7]],

       [[0.5]],

       [[0.6]],

       [[0.7]],

       [[0.1]],

       [[0.3]],

       [[0.8]],

       [[0.1]],

       [[0.2]],

       [[0.4]]])

In [87]:

model = hmm.GaussianHMM(n_components=N*d,covariance_type='spherical', startprob_prior = _inital_hlearn,
                        transmat_prior= _trans_hlearn,
                     init_params="", params="")
model.means_ = _mues_hlearn
model.covars_ = _cov_hlearn

In [88]:
# ?hmm.GaussianHMM

In [89]:
model.sample(10)

NotFittedError: This GaussianHMM instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [75]:
_mues_hlearn.shape[1]

1